# 01. Create polygon mask for Sentinel-2 and Landsat tiles

First step to create the polygon mask that is then ingested into GEE for processing water surface area time-series.

This notebook reads in:
- a Sentinel-2 image (full tile)
- a spatial layer with waterbodies' boundaries

And returns:
- a polygon mask the same size of the tile where each waterbody is labelled with a different uint16 value

#### Initial settings

In [1]:
# main libraries
import os
import numpy as np
import csv
import shutil
import zipfile

# plotting modules
import matplotlib.pyplot as plt
from matplotlib import colors

# ignore annoying warnings
import warnings
warnings.filterwarnings("ignore")

# geoprocessing modules
from skimage import morphology, transform
from skimage.draw import polygon2mask
from osgeo import gdal
from osgeo import osr
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.transform import from_origin

# few auxiliary functions
from utils import write_dict_to_csv, reproject, duplicates_dict, convert_world2pix


### Set filepaths and parameters

In [7]:
# inputs and outputs directories
fp_inputs = os.path.join(os.getcwd(),'inputs')
fp_outputs = os.path.join(os.getcwd(),'outputs')
if not os.path.exists(fp_outputs): os.makedirs(fp_outputs)

# load waterbodies layer
fp_OFS = os.path.join(fp_inputs,'waterbodies_boundaries.geojson')
gdf_OFS = gpd.read_file(fp_OFS, driver="GeoJSON")
print('%d polygons loaded'%len(gdf_OFS))

# parameters
LABEL_MULTIPLE = 5      # Multiple by which to multiply the labels
BUFFER_LANDSAT = 10     # 10m buffer to allow for georeferencing errors in Landsat
BUFFER_S2 = 20          # 20m buffer to allow for georeferencing errors in S2

# check for empty geometries
idx_empty = []
for i in range(len(gdf_OFS)):
    if gdf_OFS.iloc[i].geometry is None:
        idx_empty.append(i)
if len(idx_empty) > 0:
    gdf_OFS.drop(idx_empty, inplace=True)
print('%d empty polygons removed from OFS'%len(idx_empty))

# create a unique id for processing in GEE
gdf_OFS['GEEID'] = ['OFS_%05d'%_ for _ in np.arange(1,len(gdf_OFS)+1)]

# reset indices
gdf_OFS.index = np.arange(len(gdf_OFS))

# save new layer
gdf_OFS.to_file(os.path.join(fp_outputs,'waterbodies_boundaries_geeid.geojson'))

# load Sentinel-2 tiles
fp_tiles_S2 = os.path.join(fp_inputs,'Sentinel2_tiles')
fn_tiles_S2 = os.listdir(fp_tiles_S2)
print('%d Sentinel-2 tiles found:'%len(fn_tiles_S2))
print(fn_tiles_S2)
fp_out_tiles_S2 = os.path.join(fp_outputs,'Sentinel2_tiles_mask')
if not os.path.exists(fp_out_tiles_S2): os.makedirs(fp_out_tiles_S2)

# load Landsat tiles
fp_tiles_L = os.path.join(fp_inputs,'Landsat_tiles')
fn_tiles_L = os.listdir(fp_tiles_L)
print('%d Landsat tiles found:'%len(fn_tiles_L))
print(fn_tiles_L)
fp_out_tiles_L = os.path.join(fp_outputs,'Landsat_tiles_mask')
if not os.path.exists(fp_out_tiles_L): os.makedirs(fp_out_tiles_L)

1254 polygons loaded
0 empty polygons removed from OFS
1 Sentinel-2 tiles found:
['T55JGH']
1 Landsat tiles found:
['LC08_L1TP_091080_20231115_20231122_02_T1_B2.TIF']


### Create Sentinel-2 mask

In [8]:
# function to read the Sentinel-2 folders
def list_files_recursive(folder_path):
    all_files = []
    try:
        # Walk through the folder and its subfolders
        for root, dirs, files in os.walk(folder_path):
            # print(f"Files in '{root}':")
            for file in files:
                all_files.append(os.path.join(root, file))
    except FileNotFoundError:
        print(f"The specified folder '{folder_path}' does not exist.")
    except PermissionError:
        print(f"Permission error accessing the folder '{folder_path}'.")
    return all_files

In [10]:
for fn_tile in fn_tiles_S2:
    tile_name = fn_tile[1:]
    fp_tile = os.path.join(fp_tiles_S2, fn_tile)
    print('Processing %s ...'%tile_name)
    # find one band
    all_files = list_files_recursive(fp_tile)
    fp_image = [_ for _ in all_files if np.logical_and('_B02.jp2' in _,'IMG_DATA' in _)][0]
    # load band
    data = gdal.Open(fp_image, gdal.GA_ReadOnly)
    georef = np.array(data.GetGeoTransform())
    bands = [data.GetRasterBand(k + 1).ReadAsArray() for k in range(data.RasterCount)]
    im = bands[0]
    proj = osr.SpatialReference(wkt=data.GetProjection())
    epsg_image = int(proj.GetAttrValue('AUTHORITY',1))
    print('Image EPSG:%d'%epsg_image)
    
    # reproject vector layer to EPSG of tile
    gdf_OFS = reproject(gdf_OFS,epsg_image)
    
    # initialise mask with zeros
    im0 = np.zeros(im.shape)
    # initialise dictionary with labels
    labels = {}

    # first run the OFS layer
    for i in range(len(gdf_OFS)):
        print('\rAdding polygon %d/%d' % (i+1, len(gdf_OFS)), end='')
        label = (i+1)*LABEL_MULTIPLE
        ofs = gdf_OFS.iloc[i]
        # add label to dictionary
        labels[ofs.GEEID] = label
        # UNFORTUNATELY some polygons are multipolygons (why does this object even exist?!)
        ofs_geom = ofs.geometry
        # if len(ofs_geom.geoms) > 1: raise
        # loop through both multipolygons and assign the same label
        for k in range(len(ofs_geom.geoms)):
            # coordinates of polygon
            try:
                coords = np.array(ofs_geom.geoms[k].buffer(BUFFER_S2,mitre_limit=2).exterior.coords)
            except:
                print('polygon %s, skipped multipolygon %d'%(ofs.GEEID,k))
            # convert to image coordinates
            polygon = convert_world2pix(coords, georef)[:,[1,0]]
            # check that geometry fits inside the image, otherwise skip it
            x = polygon[:,1]
            y = polygon[:,0]
            if np.any(x<0) or np.any(y<0): continue
            if np.any(x>im0.shape[1]) or np.any(y>im0.shape[0]): continue
            # create mask (binary)
            mask_polygon = polygon2mask(im.shape, polygon)
            # multiply binary image by label
            mask_label = label*mask_polygon
            # set to 0 pixels that were already labelled
            mask_label[im0>0] = 0
            # add to mask
            im0 += mask_label
            
    # write to a tif file
    fn_image = fp_image.split('\\')[-1].replace('.jp2','.tif')
    fn_im_labelled = os.path.join(fp_out_tiles_S2, fn_image)
    # write with rasterio
    georef2 = from_origin(georef[0]-georef[1]/2, georef[3]-georef[5]/2, georef[1], georef[1])
    metadata = {
        'driver': 'GTiff',
        'count': 1,
        'dtype': 'int16',
        'width': im0.shape[1],
        'height': im0.shape[0],
        'crs': 'EPSG:%d'%epsg_image,
    'transform': georef2,
        'compress': 'lzw',
    }
    with rasterio.open(fn_im_labelled, 'w', **metadata) as dst:
        dst.write(im0, 1)
    
# write labels dict to csv
write_dict_to_csv(labels, os.path.join(fp_outputs,'labels_S2.csv'))

Processing 55JGH ...
Image EPSG:32755
coordinates are in epsg:4326
coordinates converted to in epsg:32755
Adding polygon 1254/1254

### Create Landsat mask

In [11]:
for fn_tile in fn_tiles_L:
    print(fn_tile)
    # load tile
    fp_im = os.path.join(fp_tiles_L, fn_tile)
    data = gdal.Open(fp_im, gdal.GA_ReadOnly)
    georef = np.array(data.GetGeoTransform())
    bands = [data.GetRasterBand(k + 1).ReadAsArray() for k in range(data.RasterCount)]
    im = bands[0]
    proj = osr.SpatialReference(wkt=data.GetProjection())
    epsg_image = int(proj.GetAttrValue('AUTHORITY',1))
    print('Image EPSG:%d'%epsg_image)

    # reproject vector layer to EPSG of tile
    gdf_OFS = reproject(gdf_OFS,epsg_image)
    
    # initialise mask with zeros
    im0 = np.zeros(im.shape)
    # initialise dictionary with labels
    labels = {}
    
    # first run the OFS layer
    for i in range(len(gdf_OFS)):
        print('\rAdding polygon %d/%d' % (i+1, len(gdf_OFS)), end='')
        label = (i+1)*LABEL_MULTIPLE
        ofs = gdf_OFS.iloc[i]
        # add label to dictionary
        labels[ofs.GEEID] = label
        # UNFORTUNATELY some polygons are multipolygons (why does this object even exist?!)
        ofs_geom = ofs.geometry
        # if len(ofs_geom.geoms) > 1:
        # loop through both multipolygons and assign the same label
        for k in range(len(ofs_geom.geoms)):
            # coordinates of polygon
            try:
                coords = np.array(ofs_geom.geoms[k].buffer(BUFFER_LANDSAT,mitre_limit=2).exterior.coords)
            except:
                print('polygon %s, skipped multipolygon %d'%(ofs.GEEID,k))
            # convert to image coordinates
            polygon = convert_world2pix(coords, georef)[:,[1,0]]
            # check that geometry fits inside the image, otherwise skip it
            x = polygon[:,1]
            y = polygon[:,0]
            if np.any(x<0) or np.any(y<0): continue
            if np.any(x>im0.shape[1]) or np.any(y>im0.shape[0]): continue
            # create mask (binary)
            mask_polygon = polygon2mask(im.shape, polygon)
            # multiply binary image by label
            mask_label = label*mask_polygon
            # set to 0 pixels that were already labelled
            mask_label[im0>0] = 0
            # add to mask
            im0 += mask_label
    
    # write to a tif file
    fn_im_labelled = os.path.join(fp_out_tiles_L, fn_tile)
    # write with rasterio
    georef2 = from_origin(georef[0]-georef[1]/2, georef[3]-georef[5]/2, georef[1], georef[1])
    metadata = {
        'driver': 'GTiff',
        'count': 1, 
        'dtype': 'int16',  
        'width': im0.shape[1],
        'height': im0.shape[0],
        'crs': 'EPSG:%d'%epsg_image,
        'transform': georef2,
        'compress': 'lzw',
    }
    with rasterio.open(fn_im_labelled, 'w', **metadata) as dst:
        dst.write(im0, 1)
        
# write labels dict to csv
write_dict_to_csv(labels, os.path.join(fp_outputs,'labels_Landsat.csv'))

LC08_L1TP_091080_20231115_20231122_02_T1_B2.TIF
Image EPSG:32655
coordinates are in epsg:32755
coordinates converted to in epsg:32655
Adding polygon 1254/1254